In [60]:
import requests
import base64

from PIL import Image
from io import BytesIO

from azure.identity import InteractiveBrowserCredential

import uuid

In [61]:
invoice_path = r'invoices\fa_problem.png'

with open(invoice_path, "rb") as fd:
    invoice = fd.read()
    
if invoice_path.split('.')[-1] == 'pdf':
        base64_invoice = base64.encodebytes(invoice)
        base64_invoice_str = base64_invoice.decode('utf-8')
else:
    invoice_imgs = [Image.open(invoice_path)]
    img_byte_arr = BytesIO()
    base64_invoice_str = []
    for img in invoice_imgs:
        img.save(img_byte_arr, format='PNG')
        base64_invoice_str += [base64.encodebytes(img_byte_arr.getvalue()).decode('utf-8') ]

In [62]:
my_params = {
    'invoice': base64_invoice_str, 
    'taxDocumentId': str(uuid.uuid4()),
    }

In [63]:
credential = InteractiveBrowserCredential(logging_enable=True)
scope  = 'api://656a65f2-7fca-4984-84c2-b66c9f644b24/.default'
token = credential.get_token(scope)
header = {"Authorization": "Bearer " + token.token}

In [64]:
resp = requests.get('https://mm-recogniser-demo.azurewebsites.net/ping', headers=header)
print(resp)
print(resp.text)

<Response [200]>
Recogniser app is running


In [65]:
resp = requests.post('https://mm-recogniser-demo.azurewebsites.net/recognise', json=my_params, headers=header)
print(resp)
invoice_data = resp.json()
invoice_data

<Response [200]>


{'documentTypeId': 'INV',
 'extractionStatusId': 'AWAIT_CONFIRM',
 'fieldList': [{'code': 'InvoiceId', 'value': '2020267'},
  {'code': 'InvoiceTotal', 'value': '6233.0'},
  {'code': 'SubTotal', 'value': '420.0'},
  {'code': 'TotalTax', 'value': '813.0'},
  {'code': 'TaxPointDate', 'value': '2020-11-06T00:00:00'},
  {'code': 'InvoiceDate', 'value': '2020-11-09T00:00:00'},
  {'code': 'DueDate', 'value': '2020-11-23T00:00:00'},
  {'code': 'SupplierCompanyId', 'value': '24185302'},
  {'code': 'SupplierTaxId', 'value': 'CZ24185302'},
  {'code': 'AmountDue', 'value': '6233.0'},
  {'code': 'Currency', 'value': 'CZK'}],
 'itemList': [],
 'ocrList': [{'fieldCode': None,
   'height': 46,
   'itemIndex': None,
   'left': 40,
   'order': -1,
   'page': 0,
   'taxrateIndex': None,
   'text': 'MONTPLAST',
   'top': 87,
   'width': 257},
  {'fieldCode': None,
   'height': 46,
   'itemIndex': None,
   'left': 305,
   'order': -1,
   'page': 0,
   'taxrateIndex': None,
   'text': '-',
   'top': 87,
   